In [1]:
%matplotlib inline
import cPickle as pkl
import pandas as pd
import numpy as np
import os,sys
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, GRU, SimpleRNN, TimeDistributed, Activation
from keras.utils.np_utils import to_categorical

In [3]:
dataset = pkl.load(open('dataset.pkl', 'rb'))

In [164]:
rnndata = [dataset[u] for u in dataset.keys() if "bot" not in str(u)]
rnndata = [h for h in rnndata if len(h) == 40]#150 > len(h) and len(h) > 140]
rnndata = np.array(rnndata) #+ 1

In [171]:
## Hyperparameters

# max sequence length.
maxlen = 39
vocab_size = max([max(x) for x in rnndata])+1
hidden = 25
print vocab_size

28


In [173]:
print('Splitting data...')
x_train = rnndata[:int(len(rnndata)*.9)]
y_train = to_categorical(np.array(x_train)[:,1:], num_classes=None).reshape(len(x_train),maxlen,vocab_size)
x_train = np.array(x_train)[:,:-1]
x_test = rnndata[int(len(rnndata)*.9):]
y_test = to_categorical(np.array(x_test)[:,1:], num_classes=None).reshape(len(x_test),maxlen,vocab_size)
x_test = np.array(x_test)[:,:-1]
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)

Splitting data...
(1466, 'train sequences')
(163, 'test sequences')
Pad sequences (samples x time)
('x_train shape:', (1466, 39))
('y_train shape:', (1466, 39, 28))
('x_test shape:', (163, 39))


In [174]:
print('Build model...')
model = Sequential()
model.add(Embedding(maxlen, hidden))
model.add(LSTM(hidden, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size)))
model.add(Activation(activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Build model...


In [182]:
model.fit(x_train, y_train, batch_size=1, nb_epoch=1, verbose=1)

Epoch 1/1
1466/1466 [==============================] - 169s - loss: 1.4190 - acc: 0.5948   


In [184]:
# only probabilities for push to same
model.predict(x_train[:2])[:,:,8]

array([[ 0.53190732,  0.73210365,  0.49607837,  0.37680081,  0.56965637,
         0.33259222,  0.54526728,  0.03003332,  0.49743149,  0.56810021,
         0.67079663,  0.71695769,  0.74097621,  0.75773829,  0.41169599,
         0.34631202,  0.49114072,  0.60219252,  0.52182341,  0.36765316,
         0.56983232,  0.34266227,  0.60488969,  0.62855732,  0.69140041,
         0.72843754,  0.55654669,  0.35963205,  0.3039937 ,  0.0453179 ,
         0.11452239,  0.45874083,  0.51785058,  0.5348019 ,  0.43377963,
         0.57761008,  0.49403003,  0.58711571,  0.67757785],
       [ 0.53190732,  0.73210365,  0.83282137,  0.87502134,  0.8977105 ,
         0.91134244,  0.9198283 ,  0.92523581,  0.92877382,  0.93115669,
         0.93280965,  0.93398786,  0.93484819,  0.93548977,  0.93597692,
         0.93635243,  0.93664527,  0.93687612,  0.93706012,  0.93720722,
         0.93732607,  0.93742234,  0.93750077,  0.93756491,  0.93761718,
         0.93766052,  0.9376958 ,  0.93772507,  0.93774939,  0.

In [187]:
model.evaluate(x_train[:20], y_train[:20], verbose=1, sample_weight=None)

20/20 [==============================] - 0s


[1.375880241394043, 0.60384613275527954]